# Importing necessary packages

In [30]:
import pandas as pd
import os
import numpy as np
import requests
import json
from sklearn.preprocessing import MinMaxScaler
from scipy import stats
from scipy.stats import pearsonr

In [2]:
pwd = os.getcwd()
pwd

'/home/chirag/git/ai/Benchmark/Sentence Similarity V.0'

# Reading the data set

In [3]:
data = open("./datasets/STS-SemEval/2012/MSRpar.train.tsv")
data = data.readlines()
data

['4.000\tBut other sources close to the sale said Vivendi was keeping the door open to further bids and hoped to see bidders interested in individual assets team up.\tBut other sources close to the sale said Vivendi was keeping the door open for further bids in the next day or two.\n',
 "3.750\tMicron has declared its first quarterly profit for three years.\tMicron's numbers also marked the first quarterly profit in three years for the DRAM manufacturer.\n",
 "2.800\tThe fines are part of failed Republican efforts to force or entice the Democrats to return.\tPerry said he backs the Senate's efforts, including the fines, to force the Democrats to return.\n",
 "3.400\tThe American Anglican Council, which represents Episcopalian conservatives, said it will seek authorization to create a separate group.\tThe American Anglican Council, which represents Episcopalian conservatives, said it will seek authorization to create a separate province in North America because of last week's actions.\n

# Separating Sentence 1, Sentence 2 and Score

In [4]:
df = []
for i in range(len(data)):
    df.append(data[i].split('\t'))
    
score = []
sentence1 = []
sentence2 = []
for j in range(len(df)):
    score.append(df[j][0])
    sentence1.append(df[j][1])
    b = df[j][2]
    sentence2.append(b.rstrip("\n"))

# Creating DataFrame with columns Sentence1, Sentence2, Score

In [15]:
dict = list(zip(sentence1, sentence2, score))
df = pd.DataFrame(dict, columns = ['Sentence1', 'Sentence2', 'Score'])

In [16]:
df.head()

,Sentence1,Sentence2,Score
0,But other sources close to the sale said Viven...,But other sources close to the sale said Viven...,4.000
1,Micron has declared its first quarterly profit...,Micron's numbers also marked the first quarter...,3.750
2,The fines are part of failed Republican effort...,"Perry said he backs the Senate's efforts, incl...",2.800
3,"The American Anglican Council, which represent...","The American Anglican Council, which represent...",3.400
4,The tech-loaded Nasdaq composite rose 20.96 po...,The technology-laced Nasdaq Composite Index <....,2.400


#  Calling web service using POST method with this request
### Model's
* USE (Universal Sentence Encoder)
* BERT (Bidirectional Encoder Representations from Transformers)

In [21]:
BERT_SimilarityScore = []
USE_SimilarityScore = []
use = []
bert = []

model = ["USE", "BERT"]
for y in model:
    url = 'http://192.168.0.199:5010/sentence_similarity?model='+y
    print(url)
    for i in range(len(df)):
        a = df.Sentence1[i].strip(",")
        b = df.Sentence2[i].strip(",")
        d = { "sentence1":"\'"+str([df.Sentence1[i].replace(",", "")])+"\'", "sentence2":"\'"+str([df.Sentence2[i].replace(",", "")])+"\'"}
        r = requests.post(url, data=d)
        if y == "USE":
            use.append(r.json())
        else:
            bert.append(r.json())
    
for u in range(len(use)):
    USE_SimilarityScore.append(float(use[u][0][0]["similarityScore"]))
    
for k in range(len(use)):
    BERT_SimilarityScore.append(float(bert[k][0][0]["smilarity_score"]))

http://192.168.0.199:5010/sentence_similarity?model=USE
http://192.168.0.199:5010/sentence_similarity?model=BERT


## Adding 2 new columns to the dataframe (USE_SimilarityScore & BERT_SimilarityScore) and also rescaling the Score column between 0 to 1 

In [24]:
dic = list(zip(sentence1, sentence2, score, USE_SimilarityScore, BERT_SimilarityScore))
df = pd.DataFrame(dic, columns = ['Sentence1', 'Sentence2', 'Score', 'USE_SimilarityScore', 'BERT_SimilarityScore'])

scaler = MinMaxScaler()
df['Score'] = scaler.fit_transform(df['Score'].values.reshape(-1,1))

In [26]:
df.head()

,Sentence1,Sentence2,Score,USE_SimilarityScore,BERT_SimilarityScore
0,But other sources close to the sale said Viven...,But other sources close to the sale said Viven...,0.789474,0.960873,0.978246
1,Micron has declared its first quarterly profit...,Micron's numbers also marked the first quarter...,0.736842,0.718226,0.945510
2,The fines are part of failed Republican effort...,"Perry said he backs the Senate's efforts, incl...",0.536842,0.785982,0.938160
3,"The American Anglican Council, which represent...","The American Anglican Council, which represent...",0.663158,0.846785,0.966029
4,The tech-loaded Nasdaq composite rose 20.96 po...,The technology-laced Nasdaq Composite Index <....,0.452632,0.871605,0.963640


# Computing the Pearson correlation between USE semantic similarity scores and human judgement Score.

In [32]:
pearson_coef, p_value = stats.pearsonr(df["Score"], df["USE_SimilarityScore"]) 
print("Pearson Correlation Coefficient: ", pearson_coef, "and a P-value of:", p_value)

Pearson Correlation Coefficient:  0.3751883142712514 and a P-value of: 1.756959208811763e-26


# Computing the Pearson correlation between BERT semantic similarity scores and human judgement Score.

In [33]:
pearson_coef, p_value = pearsonr(df["Score"], df["BERT_SimilarityScore"]) #define the columns to perform calculations on
print("Pearson Correlation Coefficient: ", pearson_coef, "and a P-value of:", p_value) # Results 

Pearson Correlation Coefficient:  0.19917268476135105 and a P-value of: 3.783413102206333e-08


In [34]:
df.corr()

,Score,USE_SimilarityScore,BERT_SimilarityScore
Score,1.000000,0.375188,0.199173
USE_SimilarityScore,0.375188,1.000000,0.324532
BERT_SimilarityScore,0.199173,0.324532,1.000000
